In [1]:
import pandas as pd 
import numpy as np
import psycopg2 

**Now we will create our own Database**

In [2]:
def create_database():
    conn=psycopg2.connect("host=localhost dbname=postgres user=postgres password=1234")
    conn.set_session(autocommit=True)
    cur= conn.cursor()
    
    #create database
    cur.execute("Drop database if exists bank_marketing_db")
    cur.execute("Create database bank_marketing_db")
    
    #closing connection from the default database
    conn.close()
    
    #connection to new data base created
    conn=psycopg2.connect("host=localhost dbname=bank_marketing_db user=postgres password=1234")
    conn.set_session(autocommit=True)
    cur=conn.cursor()
    
    return cur,conn

**we will read the CSVs' files and check for the Data types**

In [3]:
client=pd.read_csv("client.csv")
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [4]:
client.dtypes

id                 int64
age                int64
job               object
marital           object
education         object
credit_default    object
housing           object
loan              object
dtype: object

In [27]:
campaign=pd.read_csv("campaign.csv")
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,999,0,NaN,0,2022-May-13
1,1,1,149,999,0,NaN,0,2022-May-19
2,2,1,226,999,0,NaN,0,2022-May-23
3,3,1,151,999,0,NaN,0,2022-May-27
4,4,1,307,999,0,NaN,0,2022-May-03


In [28]:
campaign.dtypes

client_id                       int64
number_contacts                 int64
contact_duration                int64
pdays                           int64
previous_campaign_contacts      int64
previous_outcome              float64
campaign_outcome                int64
last_contact_date              object
dtype: object

In [7]:
economics=pd.read_csv("economics.csv")
economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [8]:
economics.dtypes

client_id                 int64
emp_var_rate            float64
cons_price_idx          float64
euribor_three_months    float64
number_employed         float64
dtype: object

**Now we will replace values with another one and we will change the type of some columns that will be suitable for the creation of tables in DataBase**

In [9]:
client["credit_default"]=client["credit_default"].replace("unknown",np.NaN)
client.head(20)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
5,5,45,services,married,basic_9y,NaN,no,no
6,6,59,admin.,married,professional_course,no,no,no
7,7,41,blue-collar,married,NaN,NaN,no,no
8,8,24,technician,single,professional_course,no,yes,no
9,9,25,services,single,high_school,no,yes,no


In [10]:
client['credit_default']=client['credit_default'].map({"no":False,"yes":True})
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,False,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,False,yes,no
3,3,40,admin.,married,basic_6y,False,no,no
4,4,56,services,married,high_school,False,no,yes


In [11]:
client['credit_default']=client['credit_default'].astype('bool')

In [12]:
client['housing']=client['housing'].map({"yes":True,"no":False})
client['housing']=client['housing'].replace("unknown",np.NaN)
client['loan']=client['loan'].map({"yes":True,"no":False})
client['loan']=client['loan'].replace("unknown",np.NaN)
client=client.astype({'housing':'bool','loan':'bool'})

In [30]:
campaign['previous_outcome']=campaign['previous_outcome'].map({0:False,1:True})
print(campaign['previous_outcome'].unique())

[nan False True]


In [31]:
campaign['campaign_outcome']=campaign['campaign_outcome'].map({0:False,1:True})
print(campaign['campaign_outcome'].unique())

[False  True]


In [32]:
#campaign=campaign.astype({'previous_outcome':'bool','campaign_outcome':'bool','last_contact_date':'datetime64[ns]'})
campaign=campaign.astype({'previous_outcome':'bool','campaign_outcome':'bool'})
campaign.dtypes

client_id                      int64
number_contacts                int64
contact_duration               int64
pdays                          int64
previous_campaign_contacts     int64
previous_outcome                bool
campaign_outcome                bool
last_contact_date             object
dtype: object

In [33]:
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,999,0,True,False,2022-May-13
1,1,1,149,999,0,True,False,2022-May-19
2,2,1,226,999,0,True,False,2022-May-23
3,3,1,151,999,0,True,False,2022-May-27
4,4,1,307,999,0,True,False,2022-May-03


**Calling our cursor and connection function "create_database"**

In [34]:
cur,conn=create_database()

In [35]:
cleint_data_table = ("""
CREATE  TABLE client(
id SERIAL PRIMARY KEY,
age INT ,
job TEXT ,
marital TEXT,
education TEXT ,
credit_default BOOL,
housing BOOL,
loan BOOL
);
""")
cur.execute(cleint_data_table)
conn.commit()

In [36]:
campaign_data_table = ("""
CREATE TABLE campaign(
campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE  
);
""")
cur.execute(campaign_data_table)
conn.commit()

In [37]:
economics_data_table = ("""
CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
""")
cur.execute(economics_data_table)
conn.commit()

**storing our Insertion queries inside the variables so we can make a for loop to insert our Data to the tables**

In [38]:
sql_client_insert = 'INSERT INTO client (id,age,job,marital,education,credit_default,housing,loan) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)'
sql_campaign_insert = 'INSERT INTO campaign (client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
sql_economics_insert = 'INSERT INTO economics (client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed) VALUES (%s,%s,%s,%s,%s)'

In [39]:
#insert client Data to client table

for row in client.itertuples():
   cur.execute(sql_client_insert,row[1:])
conn.commit()

In [41]:
#insert campaign Data to campaign table

for row in campaign.itertuples():
    cur.execute(sql_campaign_insert,row[1:])
conn.commit()

In [43]:
#insert economics Data to economics table 

for row in economics.itertuples():
    cur.execute(sql_economics_insert,row[1:])
    
conn.commit()

**Now we changed some Data Types and replace values so it can fit the DataBase and appending our Data to its tables**